In [85]:
import sys
import json
import argparse
import random
import csv
import numpy as np
from scipy.signal import argrelextrema
from pandas import read_csv
import random


import tensorflow as tf
import keras
import PIL
import numpy as np
import pandas as pd
import os
import math
from sklearn.model_selection import train_test_split
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Dropout, BatchNormalization
def read_file(file_path, type):
    if type == "csv":
        data = read_csv(file_path)
        return data.values
    elif type == "json":
        f = open(file_path)
        data = json.load(f)
        return data

In [96]:
session_path = 'toadstool/participants/participant_3/participant_3_session.json'
truth_path = 'truth/participant_3_events.json'
vtruth_path = 'vtruth/participant_3_video.csv'

In [97]:
    event = {0:"new_stage", 1:"flag_reached", 2:"status_up", 3:"status_down", 4:"life_lost"}
    out = {"new_stage":0, "flag_reached":1, "status_up":2, "status_down":3, "life_lost":4}
 # game frame number
    game = read_file(session_path, "json")#frame的每個數字
    game_len = int(game["obs_n"])

In [98]:
 game_len

124867

In [86]:
p0='emotrain/e0.csv'
p1='emotrain/e1.csv'
p3='emotrain/e3.csv'
p5='emotrain/e5.csv'
p6='emotrain/e6.csv'
p8='emotrain/e8.csv'
p9='emotrain/e9.csv'

In [87]:
t0 = read_file(p0, "csv")
t1 = read_file(p1, "csv")
t3 = read_file(p3, "csv")
t5 = read_file(p5, "csv")
t6 = read_file(p6, "csv")
t8 = read_file(p8, "csv")
t9 = read_file(p9, "csv")

In [88]:
t0 = pd.DataFrame(t0)
t1 = pd.DataFrame(t1)
t3 = pd.DataFrame(t3)
t5 = pd.DataFrame(t5)
t6 = pd.DataFrame(t6)
t8 = pd.DataFrame(t8)
t9 = pd.DataFrame(t9)

In [148]:
inp0='emoinput/p0_emotion.csv'
inp2='emoinput/p2_emotion.csv'
inp3='emoinput/p3_emotion.csv'
inp4='emoinput/p4_emotion.csv'
inp5='emoinput/p5_emotion.csv'
inp7='emoinput/p7_emotion.csv'

In [182]:
in0 = read_file(inp0, "csv")
in2 = read_file(inp2, "csv")
in2 = read_file(inp2, "csv")
in3 = read_file(inp3, "csv")
in4 = read_file(inp4, "csv")
in5 = read_file(inp5, "csv")
in7 = read_file(inp7, "csv")

In [255]:
temotion = pd.DataFrame(in0)

In [256]:
temotion.shape[0]

6289

In [257]:
temotion.set_index(temotion[8],inplace=True)
temotion = temotion.drop([0],axis=1)
temotion

,1,2,3,4,5,6,7,8
8,,,,,,,,
0.0,0.00,0.0,0.02,0.22,0.03,0.00,0.72,0.0
10.0,0.00,0.0,0.02,0.06,0.02,0.00,0.90,10.0
20.0,0.00,0.0,0.01,0.06,0.01,0.00,0.91,20.0
30.0,0.00,0.0,0.03,0.04,0.03,0.00,0.89,30.0
40.0,0.00,0.0,0.02,0.12,0.02,0.00,0.84,40.0
...,...,...,...,...,...,...,...,...
62870.0,0.04,0.0,0.16,0.07,0.08,0.01,0.64,62870.0
62880.0,0.02,0.0,0.16,0.17,0.11,0.01,0.51,62880.0
62890.0,0.02,0.0,0.38,0.16,0.24,0.09,0.10,62890.0


In [258]:
# Scale the data
from sklearn.preprocessing import MinMaxScaler
import numpy as ny
newframe = []
frame = list(range(temotion.shape[0]))
frame=np.array(frame)
frame = frame[:,np.newaxis]

scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(frame)

for i in scaled:
    i=float(i*game_len)
    newframe.append(round(i))

In [259]:
temotion['normalize'] = newframe
temotion.set_index(temotion['normalize'],inplace=True)
temotion = temotion.drop(['normalize'],axis=1)
#drop 2,3,7,8 angry,disgust,surprise,neutral,box
temotion = temotion.drop([2],axis=1)
temotion = temotion.drop([7],axis=1)
temotion = temotion.drop([8],axis=1)
temotion.columns = ['angry','fear','happy','sad','surprise']
temotion = temotion.drop(['angry'],axis=1)
temotion = temotion.drop(['surprise'],axis=1)
temotion

,fear,happy,sad
normalize,,,
0,0.02,0.22,0.03
20,0.02,0.06,0.02
40,0.01,0.06,0.01
60,0.03,0.04,0.03
79,0.02,0.12,0.02
...,...,...,...
124788,0.16,0.07,0.08
124807,0.16,0.17,0.11
124827,0.38,0.16,0.24


In [274]:
emotion = pd.concat([t0,t1,t3,t5,t6,t8,t9])
emotion

,0,1,2,3,4,5,6,7
0,0.0,0.01,0.04,0.07,0.06,0.00,0.0,0.0
1,2.0,0.01,0.03,0.08,0.03,0.00,1.0,0.0
2,4.0,0.01,0.04,0.05,0.05,0.00,2.0,0.0
3,6.0,0.01,0.04,0.08,0.03,0.00,3.0,0.0
4,8.0,0.00,0.04,0.07,0.02,0.00,4.0,0.0
...,...,...,...,...,...,...,...,...
62953,125373.0,0.03,0.10,0.64,0.02,0.19,62953.0,0.0
62954,125375.0,0.00,0.01,0.95,0.00,0.03,62954.0,0.0
62955,125377.0,0.00,0.03,0.89,0.00,0.07,62955.0,0.0
62956,125379.0,0.00,0.01,0.97,0.00,0.02,62956.0,0.0


In [275]:
y_train = emotion[7]
x_train =emotion.drop([6],axis=1)
x_train =x_train.drop([0],axis=1)
x_train =x_train.drop([7],axis=1)
x_train =x_train.drop([1],axis=1)
x_train =x_train.drop([5],axis=1)

In [276]:
x_train

,2,3,4
0,0.04,0.07,0.06
1,0.03,0.08,0.03
2,0.04,0.05,0.05
3,0.04,0.08,0.03
4,0.04,0.07,0.02
...,...,...,...
62953,0.10,0.64,0.02
62954,0.01,0.95,0.00
62955,0.03,0.89,0.00
62956,0.01,0.97,0.00


In [277]:
x_train.shape

(440305, 3)

In [278]:
y_train.shape

(440305,)

In [288]:
import numpy as np    
from keras.layers.convolutional import Conv1D    
from keras.layers.convolutional import MaxPooling1D   
from keras.models import Sequential     

In [ ]:
model = Sequential()
n_timesteps, n_features, n_outputs = x_train.shape[0], x_train.shape[1], y_train.shape[0]
model.add(Conv1D(input_shape=(n_timesteps,n_features),filters=64, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5) # Callback for earlystopping
model4.compile(optimizer="sgd", loss="binary_crossentropy" , metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=32, epochs=7, validation_data=(x_test, y_test), callbacks=[callback])

In [248]:
#Split the data into training and validation sets
#Build you model and evaluate it on your validation set
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

x =x_train
y =y_train
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=0)

In [240]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10)
rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=10)

In [241]:
rf_pred = rf.predict(x_test)

In [242]:
import sklearn.metrics as metrics
print("Accuracy:",metrics.accuracy_score(y_test, rf_pred))

Accuracy: 0.9978121309390425


In [243]:
choose0 = rf.predict(temotion)

In [279]:
len(y_train)

440305

In [280]:
j=0

In [281]:
for i in y_train:
    if i==1:
        j=j+1
j

960